In [1]:

using MAT
using CSV
using GraphPlot
using CSV
using LightGraphs
using DataFrames
using JLD
using Plots
using Measures
using StatsPlots
using StatsBase
using HypothesisTests
using PyCall
using RCall
using MultivariateStats
using Printf
using LinearAlgebra
sns = pyimport("seaborn")
plt = pyimport("matplotlib.pyplot")
scipy = pyimport("scipy")

#@pyimport seaborn as sns
#@pyimport matplotlib.pyplot as plt
#@pyimport scipy

R"library(TDA)"
R"library(igraph)"
R"library(ggplot2)"

R"source('~/Dropbox/Top Sim and Homog/Scripts/bottleneck_computations_functions2.R')"
R"source('~/Dropbox/Top Sim and Homog/Scripts/local_network_functions.R')"

## Pre-defined functions

## Functions

function toWeightedAdj_byweight(network_und,node_weights)
    #Assume that HIGHER node weight means born earlier-- this means we
    # want to keep the MINIMUM
    nNodes = length(node_weights)
    t0 = network_und.*node_weights
    t1 = deepcopy(t0)
    for i in collect(1:nNodes)
        for j in collect(i:nNodes)
            t1[i,j] = minimum([t0[i,j],t0[j,i]])
            t1[j,i] = t1[i,j]
        end
    end
    t1
end

# To calculate the bettiCurves from the barcodes
function bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)
    
    nNodes = Int(nNodes)
    nmats = Int(nmats)
    maxDim = Int(maxDim)
    bettiBar = zeros(nmats,maxDim)
    bettiCurve = zeros(nmats,nNodes+1,maxDim)
    birthCurve = zeros(nmats,nNodes,maxDim)
    deathCurve = zeros(nmats,nNodes,maxDim)
    

    for dimn in collect(1:maxDim)
        dimn = Int(dimn)
       
        for matn in collect(1:nmats)
            matn = Int(matn)
            bb = 0
            currentCurve = barcode_array[matn,:]
            currentCurveDim = currentCurve[dimn]
            for barn in collect(1:size(currentCurveDim,1))
               

                # Add to birth curve
                birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] = birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] .+1


                if currentCurveDim[barn,2]>nNodes
                  
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] .+1
                    bb = bb+(nNodes+1-currentCurveDim[barn,1])
                    else 
                   
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn].+1
                    deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] = deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] .+1
                    bb = bb+(currentCurveDim[barn,2] - currentCurveDim[barn,1])

                end
            end
            
            bettiBar[matn,dimn] = deepcopy(bb)
        end
    end
    
    return bettiCurve, birthCurve, deathCurve, bettiBar 
end


## Calculating edge density
function calculateEdgeQuantity(jadj_array)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    edgeQuantity = zeros(nmats,nNodes)
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            edgesAdded = (LinearIndices(adj))[findall(adj.<= noden)]
            edgeQuantity[Int(matn),Int(noden)] = length(edgesAdded)/2
        end
    end
    edgeQuantity = edgeQuantity.-35   # get rid of self-loops = 0
    
end

function calculateDegreesFiltration(jadj_array,nNodes)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    degree_array = zeros(nNodes,nNodes,nmats)
    
    jadj_array[jadj_array.==(nNodes*2)] .= 0
    
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            adj_i = deepcopy(adj)
            adj_i[adj_i.>noden] .= 0
            adj_i[adj_i.>0] .= 1
            degree_array[:,Int(noden),Int(matn)] = sum(adj_i,dims = 1)
        end
    end
    
    return degree_array
    
end


## Making weights from order
function orderToWeights(s_0_array,nNodes)
        s_wei_array = zeros(size(s_0_array))
    for i0 in collect(1:size(s_0_array,1))

        for n0 in collect(1:nNodes)
            n0 = Int(n0)
            s_wei_array[i0,n0] = indexin([n0],s_0_array[i0,:])[1]
        end
    end
    s_wei_array
end

function calculateTheoreticaMaxDistance(s_wei_array,n) 
    theo_dist= Vector{Float64}(undef,n)   #10100 or 48320
    for i1 in collect(1:size(s_wei_array,1))
        norm1 = norm(s_wei_array[1,:].-s_wei_array[i1,:],Inf)
        theo_dist[i1] = norm1
    end
    return theo_dist
end

## Making Diagrams from barcode for TDA in R
function makeDiagramFromBarcode(barcode_array,graphn,maxDim)

    diag = []
    nNodes = 70


    for d in collect(1:maxDim)
        
        bd = barcode_array[graphn,d]
        infs = findall(bd[:,2] .> nNodes)
        bd[infs,2] .= nNodes+1
        nbars = size(bd,1)
        
        if nbars>0
            bda = hcat(d*ones(nbars),bd)
        else
            bda = [d 0 0]
        end
                
                        
       
        
        if isequal(diag,[])
            diag = bda
            else
            diag = vcat(diag,bda)
        end
                
    end
    diag
end




function computeBNDistances_glob(barcode_array)

    nReps = 101
    nGraphs = 100
    maxDim = 3

    graphOriginals = collect(1:101:10100)
    distanceBN_array = zeros(10100,3)
    

    for nG in graphOriginals
               
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim)
        
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end


            
function computeBNDistances_globSampled(barcode_array)

    nReps = 101
    nGraphs = 100
    maxDim = 3
    runs = collect(1:100)

    graphOriginals = collect(1:101:10100)
    distanceBN_array = zeros(10000,3)
    

    for nG in collect(1:nGraphs)
                    
        a = (nG-1)*nReps + 1
                    
        for runi in runs
                        
        p = sample(collect(a:(a+100)),2,replace = false)
        diaga = makeDiagramFromBarcode(barcode_array,p[1],maxDim)
        diagb = makeDiagramFromBarcode(barcode_array,p[2],maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diaga,$diagb,$nD)"
                distanceBN_array[((nG-1)*(nReps-1) + runi),nD] = distanceBN_dimn
            end
        end

    end
    
    return distanceBN_array
end

            
            
            
function computeBNDistances_Sampled(barcode_array,nGraphs)

    
    maxDim = 3
    distanceBN_array = zeros(10000,3)
    
    runs = collect(1:size(distanceBN_array)[1])

    for runi in runs
        p = sample(collect(1:nGraphs),2,replace = false)
        diaga = makeDiagramFromBarcode(barcode_array,p[1],maxDim)
        diagb = makeDiagramFromBarcode(barcode_array,p[2],maxDim)

        for nD in collect(1:3)
            distanceBN_dimn = R"computeBNDistance($diaga,$diagb,$nD)"
            distanceBN_array[runi,nD] = distanceBN_dimn
        end

    end
    
    return distanceBN_array
end
            

function computeBNDistances_local(barcode_array)

    nReps = 2416
    nGraphs = 20
    maxDim = 3
    nNodes = 70

    #graphOriginals = collect(1:2416:24160)
    #distanceBN_array = zeros(24160,3)
    graphOriginals = collect(1:2416:48320)
    distanceBN_array = zeros(48320,3)

    for nG in graphOriginals
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim)
        println(nG)
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end
    
    
function plotBarcode(allPIs,nNodes,graphN,maxDim,fontSize)

    nNodes = Int(nNodes)
    graphn = Int(graphN)
    maxDim = Int(maxDim)
    counter1 = 0
    pbar = plot(1:6,zeros(6),c=:black)
    
    colors = [:blue :green :red]
    for dim in collect(1:maxDim)

        barn = barcode_array[graphN, dim]
        barn = barn[sortperm(barn[:,1]),:]

        nbars = size(barn)[1]


        for cntr1 in collect(1:nbars)
            birth = barn[cntr1,1]
            death = barn[cntr1,2]

            plot!([birth, death],[cntr1+counter1, cntr1+counter1],c=colors[dim], legend = false,
                            xlim = (0,nNodes), ytickfont = font(fontSize), xtickfont = font(fontSize))
        end

        display(pbar)



        counter1 = counter1+nbars
    end

    return pbar
end

            
function strictLTvector(x)
    xVec = [x[i,j] for j in 1:size(x)[2]-1 for i in j+1:size(x)[1]]
    return xVec
end
            
function strictLTvector3D(x)
    xVec = [x[i,j,d] for d in 1:size(x)[3] for j in 1:size(x)[2]-1 for i in j+1:size(x)[1]]
    return xVec
end    

┌ Info: Recompiling stale cache file /Users/annsizemore/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/annsizemore/.julia/compiled/v1.0/StatsPlots/SiylL.ji for StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/annsizemore/.julia/compiled/v1.0/PyCall/GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1190
┌ Warning: RCall.jl: 
│ Attaching package: ‘igraph’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     decompose, spectrum
│ 
│ The following object is masked from ‘package:base’:
│ 
│     union
│ 
└ @ RCall /Users/annsizemore/.julia/packages/RCall/ffM0W/src/io.jl:113


strictLTvector3D (generic function with 1 method)

In [2]:
graph_model_names = ["abssin_2pi","ER_p04","PropProb_d1","prefAttach_m4","RG_ep015","spatialGrowth_b1a4"]

colors_vec = [colorant"#513D5E" colorant"#3B4B82" colorant"#7e918a"]


In [16]:




for graph_model_name in graph_model_names

    #### Compute topological overlap
    graph_name_local = "NF_$(graph_model_name)_1218_local"
    dict6 = matread("Results/$(graph_name_local).mat")
    badj_array_all = dict6["badj_array"]
    nReps = 2416
    nNodes = 70
    nGraphs = 20
    maxDim = 3
    origs = collect(1:nReps:size(badj_array_all,3))
    badj_array = badj_array_all[:,:,origs]
    badj_array_all = nothing
    dict6 = nothing
    GC.gc()

    R"source('~/Dropbox/Top Sim and Homog/Scripts/local_network_functions.R')"
    ## Compute topological overlap on everything
    tolap_all = zeros(nNodes, nNodes, nGraphs)
    for i0 in collect(1:nGraphs)
        badj1 = badj_array[:,:,i0]
        R_tolap = R"calculate_top_overlap"
        tolapR = R_tolap(badj1)
        tolap = rcopy(tolapR)
        tolap_all[:,:,i0] = deepcopy(tolap)

    end

    tolap_all_vec = strictLTvector3D(tolap_all)



    ## Compute topological similarity

    pick = "new"
    dmat_local = []
    if pick == "old"
        dict5 = matread("Results/$(graph_name_local)_bottleneckDistances.mat")
        dmat_local = dict5["distanceMat"]
    else
        z1= load("Results/$(graph_name_local)_pis.jld")
        barcode_array = z1["barcode_array"]
        println("Computing bottleneck distances on local reorderings")
        dmat_local = computeBNDistances_local(barcode_array)
        z1 = nothing
    end

    println("loaded :)")
    print("loaded relevant data")
    println(nGraphs)
    println(nReps)

    dict6 = matread("Results/$(graph_name_local).mat")
    s_0_array_local = dict6["s_0_array"]
    nNodes = dict6["nNodes"]
    nNodes = Int(nNodes)
    nGraphs = dict6["nGraphs"]
    nGraphs = Int(nGraphs)
    nReps = 2416
    dict5 = nothing
    dict6 = nothing
    dict7 = nothing
    GC.gc()
    print("loaded relevant data")
    println(nGraphs)
    println(nReps)


    # Need to make an ordering to distance function
    # Will make tomorrow for now will plot
    s_wei_array_local = orderToWeights(s_0_array_local,nNodes)
    theo_dist_local = calculateTheoreticaMaxDistance(s_wei_array_local,48320) 

    ## Swap thing
    theo_dist_swap = zeros(Int(nNodes),Int(nNodes),nGraphs)
    bdist_swap = zeros(Int(nNodes),Int(nNodes),nGraphs,maxDim)
    
    println(size(badj_array))

    iter = 1
    for r in 1:nGraphs
        iter = iter+1

        for i0 in collect(1:nNodes)
            for j0 in collect((i0+1):nNodes)
                theo_dist_swap[Int(i0),Int(j0),Int(r)] = theo_dist_local[iter]
                theo_dist_swap[Int(j0),Int(i0),Int(r)] = theo_dist_swap[Int(i0),Int(j0),Int(r)]
                for d0 in collect(1:maxDim)
                    bdist_swap[Int(i0),Int(j0),Int(r),d0] = dmat_local[iter,d0]
                    bdist_swap[Int(j0),Int(i0),Int(r),d0] = bdist_swap[Int(i0),Int(j0),Int(r),d0]
                end
                iter = iter+1
            end
        end
    end

    println(size(bdist_swap))
    bdist_swap_mean1 = dropdims(mean(bdist_swap,dims = 4), dims = 4)
    println(size(bdist_swap_mean1))

    sim_matrices = 1 .- bdist_swap_mean1./theo_dist_swap
    sim_matrices_vec = strictLTvector3D(sim_matrices)
    println("Sim_matrices_vec size")
    println(size(sim_matrices_vec))
    println(size(tolap_all_vec))

    m1,b1,r_val1,p_val1,ste1 = scipy.stats[:linregress](x = sim_matrices_vec, y= tolap_all_vec)
    sns_p1 = sns.jointplot(x = sim_matrices_vec, y=tolap_all_vec, kind = "reg", height = 3)
    sns_p1[:set_axis_labels]("Topological similarity","Topological Overlap",fontsize = 10)
    plt.gca()[:annotate]("r = $(@sprintf("%.4f", r_val1)) \np = $(@sprintf("%.4E", p_val1))",xy = (minimum(sim_matrices_vec),minimum(tolap_all_vec)))
    sns_p1[:savefig]("$(graph_model_name)_sns1.png",dpi = 300)
        
        
        
        
        
        
    # plot other thing
    sim_array = 1 .- bdist_swap./theo_dist_swap
    sim_array[isnan.(sim_array)] .= 0
    sim_weighted_degree_array = dropdims(sum(sim_array, dims = 1), dims = 1)
    sim_weighted_degree_mean = dropdims(mean(sim_weighted_degree_array,dims = 3), dims = 3)
    println(size(sim_weighted_degree_mean))
    badj_degree_array = dropdims(sum(badj_array,dims = 1), dims = 1)
    println(size(badj_degree_array))
    
        
    m,b,r_val,p_val,ste = scipy.stats[:linregress](x = [badj_degree_array...], y= [sim_weighted_degree_mean...])
    sns_p2 = sns.jointplot(x = [badj_degree_array...], y = [sim_weighted_degree_mean...], kind = "reg", height = 3)
    sns_p2[:set_axis_labels]("Degree","Summed topological similarity",fontsize = 10)
    plt.gca()[:annotate]("r = $(@sprintf("%.4f", r_val)) \np = $(@sprintf("%.4E", p_val))",xy = (minimum([badj_degree_array...])-2,minimum([sim_weighted_degree_mean...])-2))
    sns_p2[:savefig]("$(graph_model_name)_sns2.png", dpi = 300)
        
    dmat = nothing
    sim_matrices = nothing
    tolap_all_vec = nothing
    badj_array_all = nothing
    GC.gc()
        
    println("Finished $(graph_model_name)")
        
end
    



    
    

Computing bottleneck distances on local reorderings
1
2417
4833
7249
9665
12081
14497
16913
19329
21745
24161
26577
28993
31409
33825
36241
38657
41073
43489
45905
loaded :)
loaded relevant data20
2416
loaded relevant data20
2416
(70, 70, 20)
(70, 70, 20, 3)
(70, 70, 20)
Sim_matrices_vec size
(48300,)
(48300,)


┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:111
└ @ Core ./In[16]:111
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:113
└ @ Core ./In[16]:113
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:114
└ @ Core ./In[16]:114
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:115
└ @ Core ./In[16]:115


(70, 20)
(70, 20)


┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:132
└ @ Core ./In[16]:132
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:134
└ @ Core ./In[16]:134
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:135
└ @ Core ./In[16]:135
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at In[16]:136
└ @ Core ./In[16]:136


Finished abssin_2pi
Computing bottleneck distances on local reorderings
1
2417
4833
7249
9665
12081
14497
16913
19329
21745
24161
26577
28993
31409
33825
36241
38657
41073
43489
45905
loaded :)
loaded relevant data20
2416
loaded relevant data20
2416
(70, 70, 20)
(70, 70, 20, 3)
(70, 70, 20)
Sim_matrices_vec size
(48300,)
(48300,)
(70, 20)
(70, 20)
Finished ER_p04
Computing bottleneck distances on local reorderings
1
2417
4833
7249
9665
12081
14497
16913
19329
21745
24161
26577
28993
31409
33825
36241
38657
41073
43489
45905
loaded :)
loaded relevant data20
2416
loaded relevant data20
2416
(70, 70, 20)
(70, 70, 20, 3)
(70, 70, 20)
Sim_matrices_vec size
(48300,)
(48300,)
(70, 20)
(70, 20)
Finished PropProb_d1
Computing bottleneck distances on local reorderings
1
2417
4833
7249
9665
12081
14497
16913
19329
21745
24161
26577
28993
31409
33825
36241
38657
41073
43489
45905
loaded :)
loaded relevant data20
2416
loaded relevant data20
2416
(70, 70, 20)
(70, 70, 20, 3)
(70, 70, 20)
Sim_matrice

# For global analyses


In [4]:
# Compute avg birth and death times.




for graph_model_name in graph_model_names

    ######## ----- ######
    makeBox = 2
    colors_orig = [RGB(0.2,0.3,0.6) RGB(0.23,0.6,0.3) RGB(0.6,0.2,0.4)]
    colors_r = [RGB(0.3,0.5,1) RGB(0.23,0.9,0.4) RGB(0.9,0.3,0.5)]

    ######## ----- ######
    graph_name_global = "NF_$(graph_model_name)_1218_glob"

    dict2 = matread("Results/$(graph_name_global).mat")
    s_0_array_glob = dict2["s_0_array"]
    nNodes = dict2["nNodes"]
    nNodes = Int(nNodes)
    nGraphs = dict2["nGraphs"]
    nGraphs = Int(nGraphs)
    nReps = dict2["nReps"]
    nReps = Int(nReps)
    maxDim = 3
 #   dict4 = matread("Results/$(graph_name_global)_pis.mat")
 #   allPIs_glob = dict4["allPersistenceIntervals"]
    z2= load("Results/$(graph_name_global)_pis.jld")
    barcode_array_glob = z2["barcode_array"]
    print(size(barcode_array_glob))
    dict2 = nothing
    dict3 = nothing
    dict4 = nothing
    GC.gc()
    print("loaded relevant data")
    println(nGraphs)
    println(nReps)





    inds1 = collect(0:nReps:(nReps*nGraphs-1))
    inds1 = inds1 .+1
    inds1 = Int.(inds1)
    barcode_array_orig = barcode_array_glob[inds1,:]
    indsr = setdiff(1:(nReps*nGraphs),inds1)
    indsr = Int.(indsr)
    barcode_array_r = barcode_array_glob[indsr,:]
       println("here1")

    ## Make betti curve and such
    bettiCurve_orig, birthCurve_orig, deathCurve_orig, bettiBar_orig = bettiCurveFromBarcode(barcode_array_orig,nNodes,nGraphs,maxDim)
    bettiCurve_r, birthCurve_r, deathCurve_r, bettiBar_r = bettiCurveFromBarcode(barcode_array_r,nNodes,size(barcode_array_r)[1],maxDim)
    
    
    # Calculate average birth/death times
    
    avg_birth_times = zeros(size(barcode_array_glob,1),3)
    avg_death_times = zeros(size(barcode_array_glob,1),3)
    for a0 in collect(1:size(avg_birth_times,1))
        for dim in collect(1:Int(maxDim))
            avg_birth = mean(barcode_array_glob[a0,dim][:,1])
            avg_birth_times[a0,dim] = avg_birth
            deathTimes = barcode_array_glob[a0, dim][:,2]
            deathTimes[deathTimes.>nNodes] .= nNodes+1
            avg_death = mean(deathTimes)
            avg_death_times[a0,dim] = avg_death
        end
    end

    # Set Nan to 0
    avg_birth_times[isnan.(avg_birth_times)] .= 0
    avg_death_times[isnan.(avg_death_times)] .= 0
    
    avg_birth_times_orig = avg_birth_times[inds1,:]
    avg_birth_times_r = avg_birth_times[indsr,:]
    avg_death_times_orig = avg_death_times[inds1,:]
    avg_death_times_r = avg_death_times[indsr,:]



    
 


    ## Calculate differences between betti bars
    bettiBarDifferences_glob = zeros(10000,3)
    avgBirthDifferences_glob = zeros(10000,3)
    avgDeathDifferences_glob = zeros(10000,3)
    
    maxDim = 3
    nGraphs = Int(nGraphs)
    for i0 in collect(1:nGraphs)

        bbar1 = bettiBar_orig[i0,:]
        abirth1 = avg_birth_times_orig[i0,:]
        adeath1 = avg_death_times_orig[i0,:]

        for r0 in collect(1:(nReps-1))

            repG = (i0-1)*nGraphs + r0
            bbar2 = bettiBar_r[Int(repG),:]
            abirth2 = avg_birth_times_r[Int(repG),:]
            adeath2 = avg_death_times_r[Int(repG),:]

            for di in collect(1:maxDim)

                bBardiff = bbar2[Int(di)].- bbar1[Int(di)]
                aBirthDiff = abirth2[Int(di)].-abirth1[Int(di)]
                aDeathDiff = adeath2[Int(di)].-adeath1[Int(di)]


                bettiBarDifferences_glob[Int(repG),di] = bBardiff
                avgBirthDifferences_glob[Int(repG),di] = aBirthDiff
                avgDeathDifferences_glob[Int(repG),di] = aDeathDiff
                
            end
        end
    end
    
    ##Plot boxplot
    gr()
    fontSize = 8
    pi_1 = violin(["BettiBar1" "BettiBar2" "BettiBar3"],bettiBarDifferences_glob, c = colors_vec, guidefont=font(fontSize), xtickfont = font(fontSize),
        ytickfont = font(fontSize), xrotation = 90, ylabel = "Difference in BettiBar", legend = false, framestyle = :box)
    pi_2 = violin(["AvgBirth1" "AvgBirth2" "AvgBirth3"],avgBirthDifferences_glob, c = colors_vec, guidefont=font(fontSize), xtickfont = font(fontSize),
        ytickfont = font(fontSize), xrotation = 90, ylabel = "Difference in Avg Birth", legend = false, framestyle = :box)
    pi_3 = violin(["AvgDeath1" "AvgDeath2" "AvgDeath3"],avgDeathDifferences_glob, c = colors_vec, guidefont=font(fontSize), xtickfont = font(fontSize),
        ytickfont = font(fontSize), xrotation = 90, ylabel = "Difference in Avg Death", legend = false, framestyle = :box)
    piall = plot(pi_1,pi_2,pi_3,layout = (1,3), size = (576,180))
    savefig("$(graph_model_name)_glob_bbar.pdf")   
    


    println("Finished $(graph_model_name)")
        
end


(10100, 3)loaded relevant data100
101
here1
Finished abssin_2pi
(10100, 3)loaded relevant data100
101
here1
Finished ER_p04
(10100, 3)loaded relevant data100
101
here1
Finished PropProb_d1
(10100, 3)loaded relevant data100
101
here1
Finished prefAttach_m4
(10100, 3)loaded relevant data100
101
here1
Finished RG_ep015
(10100, 3)loaded relevant data100
101
here1
Finished spatialGrowth_b1a4
